In [120]:
import pandas as pd
import numpy as np

In [121]:
df = pd.read_csv('Student_Performance_Data _.csv')
df.shape

(2392, 15)

In [122]:
df.drop(['StudentID', 'GradeClass'], axis='columns', inplace=True)

In [123]:
df.isnull().sum()

Age                  0
Gender               0
Ethnicity            0
ParentalEducation    0
StudyTimeWeekly      0
Absences             0
Tutoring             0
ParentalSupport      0
Extracurricular      0
Sports               0
Music                0
Volunteering         0
GPA                  0
dtype: int64

In [124]:
df['StudyTimeWeekly']=df['StudyTimeWeekly'].apply(round)

In [125]:
df['Gender'].replace([0, 1], ['Male', 'Female'], inplace=True)
df['ParentalEducation'].replace([0, 1, 2, 3, 4], ['None', 'High School', 'Some College', "Bachelor's", 'Higher'], inplace=True)
df['Ethnicity'].replace([0, 1, 2, 3], ['Caucasian', 'African American', 'Asian', 'Other'], inplace=True)
df['ParentalSupport'].replace([0, 1, 2, 3, 4], ['None', 'Low', 'Moderate', 'High', 'Very High'], inplace=True)
df['Tutoring'].replace([0, 1], ['No', 'Yes'], inplace=True)
df['Extracurricular'].replace([0, 1], ['No', 'Yes'], inplace=True)
df['Sports'].replace([0, 1], ['No', 'Yes'], inplace=True)
df['Music'].replace([0, 1], ['No', 'Yes'], inplace=True)
df['Volunteering'].replace([0, 1], ['No', 'Yes'], inplace=True)

/var/folders/d6/5nf15lhd3x938pm5m4qz6wkh0000gn/T/ipykernel_30121/2392882135.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Gender'].replace([0, 1], ['Male', 'Female'], inplace=True)
/var/folders/d6/5nf15lhd3x938pm5m4qz6wkh0000gn/T/ipykernel_30121/2392882135.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

In [126]:
df.head(3)

,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA
0,17,Female,Caucasian,Some College,20,7,Yes,Moderate,No,No,Yes,No,2.929196
1,18,Male,Caucasian,High School,15,0,No,Low,No,No,No,No,3.042915
2,15,Male,Asian,Bachelor's,4,26,No,Moderate,No,No,No,No,0.112602


In [127]:
categorical_data = ['Age','Gender','Ethnicity', 'ParentalEducation','Tutoring','ParentalSupport','Extracurricular','Sports','Music','Volunteering']
dummy = pd.get_dummies(df, columns=categorical_data, dtype=int)
dummy.head(3)

,StudyTimeWeekly,Absences,GPA,Age_15,Age_16,Age_17,Age_18,Gender_Female,Gender_Male,Ethnicity_African American,...,ParentalSupport_None,ParentalSupport_Very High,Extracurricular_No,Extracurricular_Yes,Sports_No,Sports_Yes,Music_No,Music_Yes,Volunteering_No,Volunteering_Yes
0,20,7,2.929196,0,0,1,0,1,0,0,...,0,0,1,0,1,0,0,1,1,0
1,15,0,3.042915,0,0,0,1,0,1,0,...,0,0,1,0,1,0,1,0,1,0
2,4,26,0.112602,1,0,0,0,0,1,0,...,0,0,1,0,1,0,1,0,1,0


In [128]:
x = dummy.drop('GPA', axis='columns')
y = dummy['GPA']

In [129]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=10)

In [130]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train.values,y_train.values)
lr.score(x_test.values,y_test.values)

0.9496375670688562

In [131]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), x, y, cv=cv)

array([0.95321585, 0.95141516, 0.95898926, 0.95413342, 0.95511055])

In [132]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'fit_intercept': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(x,y)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/opt/anaconda3/lib/python3.12/site-pack

,model,best_score,best_params
0,linear_regression,0.954573,{'fit_intercept': False}
1,lasso,0.824654,"{'alpha': 1, 'selection': 'random'}"
2,decision_tree,0.854411,"{'criterion': 'friedman_mse', 'splitter': 'best'}"


In [133]:
def predict_GPA(StudyTimeWeekly,Absences,Age,Gender,Ethnicity,ParentalEducation,Tutoring,ParentalSupport,Extracuricular,Sports,Music,Volunteering):
    qual_data = np.array([Age,Gender,Ethnicity,ParentalEducation,Tutoring,ParentalSupport,Extracuricular,Sports,Music,Volunteering])
    indices = []

    for arg in qual_data:
        indices.append(np.where(x.columns==arg)[0][0])
    
    data = np.zeros(len(x.columns))
    data[0] = StudyTimeWeekly
    data[1] = Absences
    for i in indices:
        data[i] = 1
    return lr.predict([data])[0]

In [134]:
from sklearn.metrics import r2_score
score = r2_score(y_test.values, lr.predict(x_test.values))
print(round(score,2)*100)

95.0


In [135]:
import pickle
with open('GPA_model.pickle','wb') as f:
    pickle.dump(lr,f)

In [136]:
import json
columns = {
    'data_columns' : [col.lower() for col in x.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))